In [1]:
#utf-8 DES block cipher

import nbimporter,os
from DES_singleblock_encrypt import des_encrypt
from DES_singleblock_decrypt import des_decrypt

class DES():
    #initialization
    def __init__(self,mode):
        self.mode = mode
        self.enc_list = list()
        self.dec_list = list()
    
    #XOR operation 16 hex digits 
    def XOR(self,hex1,hex2):
        xor_result = str()
        bit1 = bin(int(hex1,16))[2:].zfill(64)
        bit2 = bin(int(hex2,16))[2:].zfill(64)
        for index in range(len(bit1)):
            r = (int(bit1[index])+int(bit2[index])) % 2
            xor_result = xor_result + str(r)
        
        xor_result_to_hex = hex(int(xor_result,2))[2:].zfill(16)
        return xor_result_to_hex
    
    def encrypt(self,plain_txt,key_16hex,iv = None):
        '''block size = 8 bytes'''
        utf8_list = list()
        padding_str = str()
        grou_list = list()
        
        '''transform plaintext into hex string using utf-8'''
        for i in plain_txt:
            if ord(i) < 128:
            #ascii/codepoint 127 :delet
                utf8_list.append(hex(ord(i)).replace('0x','').zfill(2))
            else:
                #utf8
                utf8_cha1 = str(i.encode('utf8')).strip('b')
                utf8_cha2 = utf8_cha1.replace('\\x','')
                utf8_cha3 = utf8_cha2.strip('\'')
                utf8_list.append(utf8_cha3)
        utf8_str = ''.join(utf8_list)
        
        '''PKCS5 padding to reach multiple of 16 hex digits'''
        padding_str = utf8_str
        if len(utf8_str) % 16 != 0:
            remainder = len(utf8_str) % 16
            padding_num = (16-remainder) // 2
            for i in range(padding_num):
                padding_str += '0' + str(padding_num)
    
        '''16 hex digits a group'''
        quotient = int(len(padding_str)/16)
        for j in range(quotient):
            item = padding_str[16*j]
            for k in range(15):
                item += padding_str[16*j + (k+1)]
            grou_list.append(item)

        '''mode operation'''
        if self.mode == 'ecb':
            for plain_16hex in grou_list:
                enc = des_encrypt(plain_16hex,key_16hex)
                self.enc_list.append(enc.generate())
            return ''.join(self.enc_list)
        
        elif self.mode == 'cbc':
            #first block XOR with initializtion vector
            xor_ob = iv       
            for plain_16hex in grou_list:
                enc = des_encrypt(self.XOR(xor_ob,plain_16hex),key_16hex)
                self.enc_list.append(enc.generate())
                xor_ob = enc.generate()
            return ''.join(self.enc_list)
        
    def decrypt(self,cipher_txt,key_16hex,iv = None):
        '''block size = 8 bytes'''
        grou_list = list()
        code_point_list = list()
        bin_str = str()
        
        '''16 hex digits a group'''
        quotient = int(len(cipher_txt)/16)
        for j in range(quotient):
            item = cipher_txt[16*j]
            for k in range(15):
                item += cipher_txt[16*j + (k+1)]
            grou_list.append(item)
 
        '''mode operation'''
        if self.mode == 'ecb':
            for cipher_16hex in grou_list:
                dec = des_decrypt(cipher_16hex,key_16hex)
                self.dec_list.append(dec.generate())
        
        elif self.mode == 'cbc':
            #first block XOR with initializtion vector
            xor_ob = iv
            for cipher_16hex in grou_list: 
                dec = des_decrypt(cipher_16hex,key_16hex)
    
                #XOR operation
                xor_result = self.XOR(xor_ob,dec.generate())
                self.dec_list.append(xor_result)
                xor_ob = cipher_16hex
        
        '''2 hex digits a group in list'''
        dec_str = ''.join(self.dec_list)
        quotient = int(len(dec_str)/2)
        for i in range(quotient):
            item = dec_str[2*i] + dec_str[2*i+1]
            code_point_list.append(int(item,16))
        
        '''eliminate padding digits'''
        padding = True
        index,num_pad = len(code_point_list)-1,1
    
        while padding:
            if code_point_list[index] == code_point_list[index-1]:
                index = index-1
                num_pad = num_pad+1
            else:
                if num_pad == code_point_list[index]:
                    for i in range(num_pad):
                        code_point_list.pop()        
                padding = False
        
        '''utf8 decoding'''
        import bitstring as bit
        for j in code_point_list:
            #append 1 byte at a time
            bin_str = bin_str + bin(j)[2:].zfill(8)
    
        '''turn string type to bytes data type'''
        decode_to_bin = bit.BitArray('0b'+ bin_str)  
        decode_result = decode_to_bin.tobytes().decode('utf8')
        return decode_result
        

#-----------testing-----------------
plaintext = 'Test測試Ã cryptography'
key_16hex = '0123456789ABCDEF'
iv = '0123456789ABCDEF'

a = AES(mode='ecb')
a_cbc_cipher = a.encrypt(plaintext,key_16hex)
a_cbc_decipher = a.decrypt(a_cbc_cipher,key_16hex)

b = AES(mode='cbc')
b_cbc_cipher = b.encrypt(plaintext,key_16hex,iv)
b_cbc_decipher = b.decrypt(b_cbc_cipher,key_16hex,iv)

print({a_cbc_cipher:a_cbc_decipher})
print({b_cbc_cipher:b_cbc_decipher})

Importing Jupyter notebook from DES_singleblock_encrypt.ipynb
Importing Jupyter notebook from subkey_generation.ipynb
Importing Jupyter notebook from mangler_function.ipynb
Importing Jupyter notebook from DES_singleblock_decrypt.ipynb
{'0814C997E8DEC664EF3C94EB2368C60D56D5F8EF98EA033BFAD63803CB1E8A57': 'Test測試Ã cryptography'}
{'1959FA8F40C3127658795A65238770B57B9797B700192262810AEDDF7E261249': 'Test測試Ã cryptography'}
